# Comparing degenerations of Schubert varieties

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook accompanying our paper "Regular subdivisions, bounds on initial ideals, and categorical limits", we compute degenerations of Schubert varieties $X(B)$.  We compare the initial degenerations $in_w(I)$ and the lower bound $I^w$ (as discussed in Section 3.2) of the ideal $I = I(X_B)$ of a Schubert variety, for all Schubert varieties contained in the Grassmannians $Gr(2,), Gr(3,7)$ and $Gr(3,8)$.


Our code uses the package Oscar in version 1.11.5.
    

In [2]:
using Oscar;
using Combinatorics;
pm = Polymake;

[ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13] (cache misses: wrong dep version loaded (2))
/home/victoria/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/bin/julia: symbol lookup error: /home/victoria/.julia/artifacts/02903f8b1c5d24ff1b9ec785fc95b09933a40097/lib/libsingular_julia.so: undefined symbol: jl_arrayset, version JL_LIBJULIA_1.11


LoadError: Failed to precompile Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13] to "/home/victoria/.julia/compiled/v1.11/Oscar/jl_oExorb".

In [2]:
include("computing_Iw_Omega.jl")
include("gfan_functions.jl")

ideal_to_gfan (generic function with 3 methods)



## Schubert varieties

Let $1\leq r \leq n$ be integers and let $B = (b_1, \ldots, b_r)$ be a $r$-element list of distinct elements of $\{1,\ldots,n\}$ in increasing order. The Schubert variety is

$$
X_{B} = \{L\in \mathrm{Gr}(r,n) : p_{A}(L) = 0 \text{ for } A \nleq B\}. 
$$

If $I_{r,n}$ denotes the Plücker ideal of $\mathrm{Gr}(r,n)$, the ideal of $X_{B}$ is given by

$$
I(X_{B}) = I_{r,n} + \langle p_{A} :  A \nleq B \rangle 
$$

In the function `schubert_variety` below, we construct the Plücker ideal of $X_B$. There is an option `elim` which, if marked `true`, will eliminate the variables $p_{A}$ for  $A \nleq B$. Note that this coincides with applying the map $p_{A}\mapsto 0$ to the ideal $I_{r,n}$. In any case, this produces a realization of  $X_B$ in a smaller coordinate subspace of the ambient Plücker projective space. 


---

**Function:** `partial_order_lists(A, B)`

*Description*: Given two vectors `A` and `B` of integers, returns `true` if both vectors have the same length and, coordinate-wise, all entries of `A` are smaller than all entries of `B`

---

**Function:** `lex_index(r, n)`

*Description*: For two integers `r` $<$ `n`, the function computes a dictionary assigning to each subset of size `r` of the integers $[n]$ its index in the lexicographical ordering of the subsets. 

---

**Function:** `remove_vars(I, vars_to_keep_index)`

*Description*: For a polynomial ideal `I` over the polynomial ring $K[x_1, \dots, x_n]$, returns the projection of `I` to the polynomial ring $K[x_1, \dots, x_m]$ where the variables in $K[x_1, \dots, x_m]$ are indexed by the vector `vars_to_keep_index`

---

**Function:** `schubert_variety(r, n, B, elim=false)`

*Description*: For two integers `r` $<$ `n` and a vector of integers `B` of length `r` with distinct entries up to `n`, computes the ideal of the Schubert variety $X_B$ as a subvariety of the Grassmannian $Gr(r,n)$. This means that the ideal is given in Plücker coordinates. There is an option `elim` which, if marked `true`, will eliminate the variables $p_{A}$ for  $A \nleq B$.

In [3]:
function partial_order_lists(A, B)
    if length(A) != length(B)
        return false
    elseif all([A[i] <= B[i] for i in 1:length(A)])
        return true
    else
        return false
    end
end

function lex_index(r, n)
    A = sort!(subsets(n,r))
    return Dict(A[i] => i for i in 1:length(A))
end

function remove_vars(I, vars_to_keep_index)
    R = base_ring(I)
    x = gens(R)
    m = length(vars_to_keep_index)
    vars_to_keep = x[vars_to_keep_index]
    S, y = polynomial_ring(coefficient_ring(R), ["y$(i)" for i in 1:m])
    phi = hom(S, R, vars_to_keep)
    return preimage(phi, I)
end

function schubert_variety(r, n, B, elim=false)
    I = [A for A in subsets(n, r) if !partial_order_lists(A, B)]
    sort!(setdiff( subsets(n, r), I))
    ncr = binomial(n,r) 
    R, x = polynomial_ring(QQ, ncr)
    Gr = grassmann_pluecker_ideal(R, r, n)
    S = base_ring(Gr)
    R = forget_grading(S)
    phi = hom(R, S, gens(S))  
    Gr = preimage(phi, Gr)
    x = gens(R)
    ind = lex_index(r, n)
    if length(I) != 0
        J = [x[ind[A]] for A in I]
    else
        J =[]
    end
    Jc = sort!([ind[A] for A in keys(ind) if !(A in I) ])
    K = Gr + ideal(R, J)
    if elim
        if length(J) != 0
            Kelim = eliminate(K, J)
        else
            Kelim = K
        end
        return remove_vars(Kelim, Jc)
    else
        return K
    end
end

schubert_variety (generic function with 2 methods)

*Example*: We compute ideal of the Schubert variety $X_{2,4,7} \subseteq Gr(3,7)$.

In [ ]:
r=3; n=7; B=[2,4,7];
IS = schubert_variety(r, n, B)

## Point configurations for Schubert varieties

In this section we compute the point configurations of Schubert varieties and Grassmannians. 

The point configuration of the Grassmannian $Gr(r,n)$ consists of the vertices of the hypersimplex $\Delta(r,n)$, and these vertices are labeled by the $r$ element subsets of $\{1,\ldots,n\}$, and the point configuiration of the Schubert variety $X_B$ consists of those vertices corresponding to the subsets $A$ satisfying $A\leq B$.


---

**Function:** `grassmannian_lineality_space(r, n)`

*Description*: Computes the point configuration of the Grassmannian of rank `r` and size `n`. 

---

**Function:** `schubert_lineality_space(r, n, B)`

*Description*: Computes the point configuration of the Schubert variety $X_B$ contained in the Grassmannian $Gr(r,n)$.

In [4]:
function grassmannian_lineality_space(r, n)
    L = zeros(Int, n, binomial(n,r))
    j=1
    for A in sort!(subsets(n, r))
        L[A, [j]] = ones(Int, r, 1)
        j+=1
    end
    return L
end

function schubert_lineality_space(r, n, B)
    I = [A for A in sort!(subsets(n, r)) if partial_order_lists(A, B)]
    L = zeros(Int, length(I), n)
    j=1
    for A in I
        L[[j], A] = ones(Int, 1, r)
        j+=1
    end
    return L
end

schubert_lineality_space (generic function with 1 method)

*Example*: We compute the point configuration of the Schubert variety $X_{2,4,7} \subseteq Gr(3,7)$.

In [ ]:
point_conf = schubert_lineality_space(r, n, B)

## The lower bound $I_w$ and the fan $\Omega(I)$

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$ from the paper, which can be computed for Grassmannians and Schubert varieties using the functions in the preceeding section. A vector $\mathsf{w}\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{\mathsf{w}}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{\mathsf{w}}(\Delta)\}$$

and the ideal they generate in $S$ is

$$I_{\mathsf{w}} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

---

For an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{\mathsf{w} \in \mathbb{R}^{n} \, : \, \mathrm{in}_{\mathsf{w}} I = I_{\mathsf{w}} \}. $$

This is a subfan of the secondary fan $\mathrm{Sec}(\Delta)$. 

---

**Function:** `Omega_fan(I, Delta, Sec)`

*Description*: Given an ideal `I` of a polynomial ring, `Delta` the point configuration of `I`, and `Sec` the secondary fan of `Delta`, returns the fan $\Omega(I)$ as a pair of rays and cones. 

Here, `Sec` can be given as a `PolyhedralFan` or as a pair `[rays_Delta, cones_Delta]` where `rays_Delta` is a `Vector{RayVector}` containing the rays and `cones_Delta` is a `IncidenceMatrix` recording the cones of `Sec`. The latter is useful when `Sec` is computed up to symmetry. 

Additional *examples* for this function can be found in `Computing_I_w_and_Omega.ipynb`.

In [6]:
function ray_cone_to_reps(rs, cs)
    n_cones, n_rays = size(cs)
    d = length(rs[1])
    reps = []
    for i in 1:n_cones
        w = [rs[j] for j in 1:n_rays if cs[i,j]]
        if length(w) == 0
            w = zeros(Int, d)
        else
            w = sum(w)
        end
        push!(reps, w)
    end
    return reps
end

function Omega_fan(I, Delta, Sec)
    if typeof(Sec) == PolyhedralFan{QQFieldElem}
        rays_Delta = rays_modulo_lineality(Sec)[1];
        cones_Delta = cones(Sec);
    else
        rays_Delta, cones_Delta = Sec
    end

    n_cones_Delta, n_rays_Delta = size(cones_Delta)
    reps = ray_cone_to_reps(rays_Delta, cones_Delta)

    tests = [true]
    for w in reps[2:length(reps)]
        w = -Vector{Int64}(pm.common.primitive(w))
        init_w_I = initial(I, nu_t, w)
        I_w = ideal_w(I,  w, Delta)
        push!(tests, init_w_I == I_w)
        #@show tests
    end
    inside_Omega = [i for i in 1:length(reps) if tests[i]];
    return [rays_Delta, cones_Delta[inside_Omega, :]]  
end

Omega_fan (generic function with 2 methods)

*Example*: We compute the fan  $\Omega(X_{2,4,7})$ using our computations of the ideal of the Schubert variety and its lineality space above, and the pre-computed tropical variety to source weight vectors. Further, we demonstrate our file management for the remainder of the notebook.

The tropicalization of $X_B$ is pre-computed as `schubert_r3_n7_B247.trop` and loaded from the file path `schubert/tropicalizations_nocones` provided in the repository. The resulting computed fan is then saved as `Omega_tropical_no_saturation_cones_r3_n7_B247.mrdi` in the folder `schubert/omega_in_tropical_fan`.

In [1]:
Bs = join([string(a) for a in B])
just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
file_name = joinpath("schubert", "tropicalizations_nocones", just_file)
rs, cs = parse_tropical_file(file_name, true)
ray_cone_to_reps(rs, cs)
O = Omega_fan(IS, point_conf, [rs, cs])
@show O
#@show cs;

LoadError: UndefVarError: `parse_tropical_file` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [33]:
just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
file_name = joinpath("schubert", "omega_in_tropical_fan", just_file)
save(file_name, O[2]);

## Comparing $in_w I$ and $I_w$

The main goal of this notebook is to analyze computationally for which Schubert varieties $X_B$ and $w \in \mathrm{Trop}(X_B)$ we have $in_w I = I_w$ where $I = I(X_{B})$. In particular, we compare $\mathrm{Trop}(X_B)$ and $\Omega(X_B)$. 

From [Cor] we know that for each Schubert variety $X_{B}$ with ideal $I = I(X_{B})$ in $Gr(2,n)$, $Gr(3,6)$ and $Gr(3,7)$, we have that the saturations of $in_{w}I^{\mathrm{sat}} = I_w^{\mathrm{sat}}$, i.e., that the very affine tropicalization and the very affine fan $\Omega(X_B)$ coincide. 

We now investigate the unsaturated case. The fans $\Omega(X_B)$ are computed as in the example above. While the computation of each fan only takes about two minutes, there are, in general, many Schubert varieties $X_B$. Thus, for the sake of running time when running this notebook, we only include the case $r =2$, $n =7$. 

For all other ranks and dimensions, computations proceed exactly analogous and can thus be easily amended. We further provide all intermediate steps and computational results in the folder `schubert` in this github repository, here. 

We start by computing the ideals of all Schubert varieties in $Gr(2,7)$ using `schubert_variety`, and save all nontrivial ideals in a format compatible for reading into `gfan` using the function `ideal_to_gfan`. The resulting files are saved in the folder `schubert/initial_files`.

In [ ]:
r = 2; n =7;# B = [4,6];
for B in sort!(subsets(n,r))
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S) 
        continue
    end
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    file_name = joinpath("schubert", "initial_files", just_file)
    println(file_name)
    ideal_to_gfan(S, true, file_name)
end

schubert/initial_files/schubert_r2_n9_B29.dat
schubert/initial_files/schubert_r2_n9_B39.dat
schubert/initial_files/schubert_r2_n9_B49.dat
schubert/initial_files/schubert_r2_n9_B59.dat
schubert/initial_files/schubert_r2_n9_B69.dat
schubert/initial_files/schubert_r2_n9_B79.dat
schubert/initial_files/schubert_r2_n9_B89.dat


To compute the tropical varieties of all of the Schubert varieties we just computed, we want to use the `gfan` algorithm `tropicaltraverse`. This requires an intermediate pre-processing step: We compute starting Groebner cones for all ideals arising from Schubert varieties using the function `gfan_tropicalstartingcone`. These are saved in the folder `schubert/start_cones`.

In [10]:
r = 2; n = 7; #B = [3,4];
for B in sort!(subsets(n,r))
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("schubert", "initial_files", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    out_file_name = joinpath("schubert", "start_cones", out_just_file)
    @show B
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicalstartingcone`, stdin=inp, stdout=out))
        end
    end
end

I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [1, 4], [2, 4], [3, 4], [1, 3], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [1, 4], [2, 4], [3, 4], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2], [1, 3]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [2, 4], [3, 4], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2], [1, 3], [1, 4]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [2, 5], [3, 5], [4, 5], [2, 4], [3, 4], 

Now we are ready to compute the tropicalizations of all Schubert varieties $X_B \subseteq Gr(2,7)$ using `gfan_tropicaltraverse`. Note that the following computation may take a few minutes. 

In [404]:
r = 2; n = 7; #B = [3,4];
for B in sort!(subsets(n,r))
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("schubert", "start_cones", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    out_file_name = joinpath("schubert", "tropicalizations", out_just_file)
    
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicaltraverse`, stdin=inp, stdout=out))
        end
    end
end

schubert/initial_files/schubert_26_156.dat
schubert/initial_files/schubert_26_256.dat
schubert/initial_files/schubert_26_356.dat
schubert/initial_files/schubert_26_456.dat
schubert/initial_files/schubert_26_146.dat
schubert/initial_files/schubert_26_246.dat
schubert/initial_files/schubert_26_346.dat
schubert/initial_files/schubert_26_136.dat
schubert/initial_files/schubert_26_236.dat
schubert/initial_files/schubert_26_145.dat
schubert/initial_files/schubert_26_245.dat
schubert/initial_files/schubert_26_345.dat
schubert/initial_files/schubert_26_135.dat
schubert/initial_files/schubert_26_235.dat


This concludes our computation of the tropicalizations. Now, we move on to compute the fan $\Omega(X_B)$ for all Schubert varieties $X_B \subseteq Gr(2,7)$, as we have done previously for $\Omega(X_{2,4,7}).

In [1]:
r = 2; n = 7; 
for B in subsets(n,r)
    IS = schubert_variety(r, n, B, true)
    if iszero(IS)
        continue
    end
    point_conf = schubert_lineality_space(r, n, B)
    Bs = join([string(a) for a in B])
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    file_name = joinpath("schubert", "tropicalizations", just_file)
    rs, cs = parse_tropical_file(file_name, true)
    ray_cone_to_reps(rs, cs)
    O = Omega_fan(IS, point_conf, [rs, cs])
    just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
    file_name = joinpath("schubert", "omega_in_tropical_fan", just_file)
    save(file_name, O[2]);
end


LoadError: UndefVarError: `subsets` not defined in `Main`
Suggestion: check for spelling errors or missing imports.